### **ESTIMATING AQUEOUS SOLUBILITY FROM MOLECULAR STRUCTURE**
##### **REF: doi.org/10.1021/ci034243x**  
---
"We have probably seen the application of machine learning in one form or another. For instance, machine learning have been used together with computer vision in self-driving cars and self-checkout convenience stores, in retail for market basket analysis (i.e. finding products that are usually purchased together), in entertainment for recommendation systems and the list goes on."

#### We will understand how machine learning could be used within the drug discovery pipeline, and in particular we will see a step-by-step procedure to build a simple regression model in Python for predicting the solubility of molecules.
---

#### **1.0 - Importing data from database**
Chemical structures are encoded by a string of text known as the 
SMILES notation which is an acronym for Simplified Molecular-Input Line-Entry System. 
Let’s have a look at the contents of the SMILES column from the sol dataframe.
Each line represents a unique molecule. 
To select the first molecule (the first row), type sol.SMILES[0] and the output that we will see is ClCC(Cl)(Cl)Cl.

#### **1.1 Converting a molecule from the SMILES string to rdkit object** 

In [1]:
#Clean the database
#Remove identical molecules


#### **1.2 Calculate molecular descriptors**
---------------------------------
We will now represent each of the molecules in the dataset by a set of molecular descriptors that will be used for model building.

To predict the aqueous solubility, we will use 4 molecular descriptors:

**MW (Molecular weight)**

**RB (Number of rotatable bonds)**

**cLogP (Octanol-water partition coefficient)**

**AP (Aromatic proportion = number of aromatic atoms / number of heavy atoms)**

Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, we will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute.

In [3]:
#Calculating LogP, MW and RB descriptors
#----------------------------------------
#We will now create a custom function called generate() for computing the 3 descriptors LogP, MW and RB.

Descriptors.MolLogP(mol)
Descriptors.NumRotatableBonds(mol)
Descriptors.MolWt(mol)
Descriptors.HeavyAtomCount(mol)



In [ ]:
#Compute SMILES = 'COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21'
m = Chem.MolFromSmiles(SMILES)
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms

def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

#Now, apply the AromaticAtoms() function to compute the number of aromatic atoms for a query SMILES string.
AromaticAtoms(m)



,MolLogP,MolWt,NumRotatableBonds
0,2.59540,167.850,0.0
1,2.37650,133.405,0.0
2,2.59380,167.850,1.0
3,2.02890,133.405,1.0
4,2.91890,187.375,1.0
...,...,...,...
1139,1.98820,287.343,8.0
1140,3.42130,286.114,2.0
1141,3.60960,308.333,4.0
1142,2.56214,354.815,3.0


#### **2.0 - DATA SPLIT**

We will now proceed to performing data splitting using a split ratio of 80/20 (i.e. we do this by assigning the test_size parameter to 0.2) whereby 80% of the initial dataset will be used as the training set and the remaining 20% of the dataset will be used as the testing set.

In [ ]:
from sklearn.model_selection import train_test_split
Y = dataset.iloc[:,1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2)

#### **3.0 - REGRESSION AND CORRELATION COEFFICIENTS**

The trained model will be applied here to predict the LogS values in the **train** and **test** set

In [ ]:
#TRAINfrom sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
model = linear_model.LinearRegression()

Y_pred_train = model.predict(X_train)
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_train, Y_pred_train))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_train, Y_pred_train))

Coefficients: [-0.75547434 -0.00649318  0.00520505 -0.35570096]
Intercept: 0.24916520303844703
Mean squared error (MSE): 0.97
Coefficient of determination (R^2): 0.77


/Users/gianvitograsso/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Let’s analyse the above output line by line:
In the first line, Coefficients lists the regression coefficient values of each independent variables (i.e. the 4 molecular descriptors consisting of LogP, MW, RB and AP)
In the second line, Intercept is essentially the y-intercept value where the regression line passes when X = 0.
In the third line, Mean squared error (MSE) is used as an error measure (i.e. the lower the better).
In the fourth line, Coefficient of determination (R²) is the squared value of Pearson’s correlation coefficient value and is used as a measure of goodness of fit for linear regression models (i.e. the higher the better)

In [ ]:
# TEST SET
Y_pred_test = model.predict(X_test)
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_test, Y_pred_test))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_test, Y_pred_test))

Coefficients: [-0.75547434 -0.00649318  0.00520505 -0.35570096]
Intercept: 0.24916520303844703
Mean squared error (MSE): 1.15
Coefficient of determination (R^2): 0.75


/Users/gianvitograsso/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


#### **5.0 - DERIVING THE LINEAR EQUATION**

In [ ]:
yintercept = '%.2f' % model.intercept_
LogP = '%.2f LogP' % model.coef_[0]
MW = '%.4f MW' % model.coef_[1]
RB = '%.4f RB' % model.coef_[2]
AP = '%.2f AP' % model.coef_[3]
print('LogS = ' + 
      ' ' + 
      yintercept + 
      ' ' + 
      LogP + 
      ' ' + 
      MW + 
      ' ' + 
      RB + 
      ' ' + 
      AP)

LogS =  0.25 -0.76 LogP -0.0065 MW 0.0052 RB -0.36 AP
